In [8]:
# Import required libraries
import numpy as np
import pandas as pd
import json
from openai import OpenAI
from pathlib import Path

In [2]:
client = OpenAI(api_key="sk-5fb8c6b1d15f4ab1bb4e218980869d8c", base_url="https://api.deepseek.com")

In [ ]:
def facts_to_pitch(fact_dict, input_category_selection):
    # fact_dict to pitch. Select input_category to feed to pitch LLM
    # ['facts', 'product_description', 'pitch_summary', 'initial_offer', 'final_offer']
    selected_inputs = {k: v for k,v in fact_dict.items() if k in input_category_selection}
    response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
            {"role": "system",
            "content":
    """
    You are a successful startup entrepreneur. You will be given a new product with some facts and description. You will create a new product pitch to raise capital to investors. You will also need to give an initial offer to the investors where you must raise as much equity as possible while minimizing the stake given. 
    Return your response in the following json format:
    {"pitch": ,
    "initial_offer: } 
    """},
            {"role": "user", "content": str(selected_inputs)},
        ],
        stream=False
    )
    response = response.choices[0].message.content
    response = json.loads(response.strip("```json\n").strip("```"))
    return response

def pitch_to_shark_final_offer(pitch_dict):
    # pitch_dict from pitch LLM to shark final offer
    response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
            {"role": "system",
            "content":
            """
    You are a successful VC investor. A startup entrepreneur is pitching a new product to you to raise capital. You have to decide whether the product is worth investing in. If so, you need to propose a final offer which will give yourself more profits or upside potential.
    Return your response in the following json format:
    {"decision to invest":
    "final_offer (if any)": {
        "Amount": ,
        "Equity": ,
        "Terms (if any)":
        }
    } 
    """},
            {"role": "user", "content": str(pitch_dict)},
        ],
        stream=False
    )    
    response = response.choices[0].message.content
    response = json.loads(response.strip("```json\n").strip("```"))
    return response

def jugde_LLM_scoring(fact_dict, final_offer_dict):
    # to give a score of how good the final offer is compared to the facts of the product
    selected_inputs = {k: v for k,v in fact_dict.items() if k in ['facts', 'product_description']}

    response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
            {"role": "system",
            "content":
            """
    You are a successful VC investor. You are observing a round of VC funding by a startup entrepreneur who is pitching a new product and a VC investor who has made a final offer. Your job is to evaluate how good the terms of offer are for the startup entrepreneur. Please think through step by step and explain your reasoning before giving a score on a scale of 1 to 10, where 10 means that the deal is very good for the entrepreneur whereas 1 means the deal is bad for the entrepreneur.
    
    Return your response in the following json format:
    {"reasoning": ,
    "rating of final_offer": } 
    """},
            {"role": "user", "content":
    f"""
    Here is the product information: {str(selected_inputs)}

    Here is the final_offer: {final_offer_dict}
    """
             },
        ],
        stream=False
    )    
    response = response.choices[0].message.content
    response = json.loads(response.strip("```json\n").strip("```"))
    return response
    

In [53]:
with Path('./all_processed_facts.txt').open("r", encoding="utf-8") as f:
        facts_store = json.loads(f.read())

scenario_folder = './scenario_basic_deepseek_R1'
fact_path = 'facts_shark_tank_transcript_28_HummViewer.txt'
scenario_name = "_".join(fact_path.split('_')[1:])

fact_dict = facts_store[fact_path]
pitch_dict = facts_to_pitch(fact_dict, ['facts', 'product_description'])
shark_final_offer_dict = pitch_to_shark_final_offer(pitch_dict)
judge_score = jugde_LLM_scoring(fact_dict, shark_final_offer_dict)
judge_score_baseline = jugde_LLM_scoring(fact_dict, fact_dict['final_offer'])
scenario_dict = {
    'fact_dict': fact_dict,
    'pitch_dict': pitch_dict,
    'shark_final_offer_dict': shark_final_offer_dict,
    'judge_score': judge_score,
    'judge_score_baseline': judge_score_baseline
}

with open(f'{scenario_folder}/{scenario_name}.txt', "w", encoding="utf-8") as file:
            json.dump(scenario_dict, file, indent=4)


In [ ]:
# @gan TODO
fact_dict = facts_store[fact_path]
pitch_dict = facts_to_pitch(fact_dict, ['facts', 'product_description'])
shark_questioning_1 = pitch_to_shark_qns1(pitch_dict)
pitch_response_1 = shark_qns1_to_response1(shark_questioning_1)
shark_questioning_2 = pitch_to_shark_qns1(pitch_dict)
pitch_response_2 = shark_qns1_to_response1(shark_questioning_1)
shark_questioning_3 = pitch_to_shark_qns1(pitch_dict)
pitch_response_3 = shark_qns1_to_response1(shark_questioning_1)
shark_final_offer_dict = pitch_to_shark_final_offer(pitch_dict)
judge_score = jugde_LLM_scoring(fact_dict, shark_final_offer_dict)
judge_score_baseline = jugde_LLM_scoring(fact_dict, fact_dict['final_offer'])